In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
from torchvision import transforms

In [2]:
#定义输入为 3*64*64 的模型
class myNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnnPart = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=3,stride=2,dilation=1), #31*31
            nn.MaxPool2d(kernel_size=3,stride=2), #15*15
            nn.ReLU(),

            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=15),
        )
        self.fcPart = nn.Sequential(
            nn.Linear(32,1), #由于是二分类任务，所以输出维度为1
            nn.Sigmoid()  #输出值在0-1之间
        )
    def forward(self,x):
        batchsize = x.shape[0]
        out = self.cnnPart(x)
        out = out.view(batchsize,-1)
        out = self.fcPart(out)
        return out

In [3]:
#定义数据集
class myDataset(Dataset):
    def __init__(self, args, type, number): #用type定义选择的test/train文件分类 #number定义文件内label=0的数量（1-58=0，>58=1)
        super().__init__()
        self.number = number
        self.type = type
        self.dataAmount = args
        self.data = torch.randn(self.dataAmount, 3,64,64)
        self.label = torch.randint(low=0,high=3,size=(self.dataAmount,1))
    def __len__(self):
        return self.dataAmount
    def __getitem__(self, item): 
        img = cv2.imread(f"C:/Engineering/dataset/{self.type}/{item+1}.jpg") #根据train/test type 选择路径 #item starts from 0, so when leading to actual image, number should be item+1
        array = cv2.resize(img,(64,64)) #cv2 格式
        transform = transforms.ToTensor() #用transform to tensor 将cv2的gbr通道改为rgb通道
        data= transform(array) 
        
        if item < self.number:   #label的赋值，由item决定
            label = torch.tensor(0)
        else:
            label = torch.tensor(1)
        return data, label
    
trainSet = myDataset(168, "train", 58) #根据myDataset init 定义的参数
testSet = myDataset(41, "test", 14)

trainLoader = DataLoader(trainSet, batch_size=32, shuffle=True)
testLoader = DataLoader(testSet, batch_size=16, shuffle=False)

In [4]:
#test#
for data,label in trainLoader: #测试trainloader输出
    print(data.shape,label.shape)
    break

torch.Size([32, 3, 64, 64]) torch.Size([32])


In [5]:
#model training
device = 'cuda:0' #在GPU运行，要注意各项（modeL，dataset，dataloader)都储存于同一位置 #当出现CUDA Error时，重启pycharm
model = myNet()
model.to(device=device)

lossfun = nn.BCELoss() #二分类任务--BCELoss,输出维度为1

optimizer = optim.Adam(model.parameters(), lr=0.0001) #优化func

In [ ]:
#training iteration
Epoch = 100 #20个大循环训练
loss_data = []
epoch_data = []
for e in tqdm(range(1, Epoch+1)):
    model.train() #start training
    for data,label in trainLoader: #iteration in test set for 1 epoch
        data = data.to(device = device)
        label = label.to(dtype = torch.float, device = device) #below when compare label&predict, predict is value between 0-1, so the data type of label must also be float, same as dtype of predict
        label = label.view(-1, 1) #增加维度，与输出(32, 1)保持一致
        output = model(data)
        loss = lossfun(output,label)
    
        #固定写法
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()       
        
        loss = loss.detach()
        loss = loss.cpu().numpy()
        loss_data.append(loss)
    #test the testdata    
    #每大循环计算一次准确率
    model.eval() #start testing
    correct = 0
    iteration = 0
    for data,label in testLoader:
        '''Prediction and accuracy'''
        data = data.to(device = device)  #testLoader, trainLoader 都要在gpu
        label = label.to(dtype = torch.float, device = device)
        label = label.view(-1,1)
        
        predict = model(data)
        predict = predict.detach() #when compare with label, label has no gradient, remove gradient of gradient as a tensor to keep them consistent
        predict = predict>0.5 #when predict >0.5, predict =1. when predict<0.5, predict =0
        result = (predict==label) #output:tensor of [True] and [False]s after compare the elements individually
        correct += torch.sum(result) #add up the tensor to give 1/0 and accumulate(+=) to work our correctness 
        #print(correct)
        # 
        # accuracy = float(correct *100)/41
        # iter_data.append(accuracy)
        # print(f"accuracy:{accuracy}%")
        
    accuracy = float(correct *100)/41  #divided by data amount for each epoch  
    epoch_data.append(accuracy)
    print(f'model training finished, Epoch:{Epoch}, ' 
          f'testSet final accuracy:{accuracy}%'
          )

#print(loss_data, epoch_data)

In [7]:
#create and write in excel
loss_list = pd.DataFrame({'Loss in each iteration': loss_data })
epoch_list = pd.DataFrame({'Accuracy in each epoch': epoch_data })
with pd.ExcelWriter("E:\Internship\Model_data.xlsx") as writer:
    loss_list.to_excel(writer, sheet_name = 'Loss', index = False)
    epoch_list.to_excel(writer, sheet_name = 'Accuracy', index = False)


In [ ]:
#test#
predict = torch.randn(16,1)
label = torch.randn(16,1)
result = (predict>0.5)
print(result==label)

In [9]:
#test working of reading img
image = cv2.imread('C:/Engineering/dataset/train/1.jpg')
 
if image is not None:
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print('图像读取失败')


In [ ]:
#test cv2#
for item in range(41):
    img = cv2.imread("C:/Engineering/dataset/test/{}.jpg".format(item+1))
    #array = np.array([img])
    array = cv2.resize(img,(64,64))
    print(array)